# 16. Preprocesado: revisión de la clase YSO

In [1]:
# Configuración general para evitar errores de warnings y compatibilidad
import warnings
import os
warnings.filterwarnings("ignore")
os.environ["RICH_NO_RICH"] = "1"
print("Configuración de entorno aplicada.")

Configuración de entorno aplicada.


## NUEVO PREPROCESADO GUARDANDO IDs DE OBJETOS

Luego se pueden usar para filtrar curvas dudosas que no están clasificando bien, y así depurar el entrenamiento.

In [ ]:
import warnings
import numpy as np
import sys
from pathlib import Path

# Añadir la raíz del proyecto al path
ROOT_DIR = Path.cwd().parent  # <- sube un nivel para alcanzar la raíz del proyecto
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

# Ignorar solo los RuntimeWarning de numpy (como overflows en reduce)
warnings.filterwarnings("ignore", category=RuntimeWarning, module="numpy")

from src.fase2.script_1_transformer_preprocessing_optimizado_2 import main as preprocessing_optimized_with_features

max_per_class_override={
    "Irregular": 9000,
    "Rotational": 9000,
    "Eclipsing Binary": 9000,
    "Delta Scuti": None,            # 7.550 → TODAS
    "RR Lyrae": 9000,               # 41.208 → TODAS NO
    "Young Stellar Object": None,   # 9.809 → TODAS
    "Cataclysmic": None,            # 2.080 → TODAS
    "White Dwarf": 0,               # 0 → LA ELIMINAMOS
    "Variable": 0                   # 0 → LA ELIMINAMOS
}

preprocessing_optimized_with_features(
    seq_length=25000,
    max_per_class=None, # usamos override completo
    max_per_class_override=max_per_class_override,
    parquet_batch_size=10_000_000,
    dataloader_batch_size=128,
    num_workers=20,
    filtrar_curvas_malas=None  # ← NUEVO
    #errores_csv_path=Path("../outputs/errores_mal_clasificados.csv")
)

📂 Cargando datos en lotes con PyArrow...
💾 [INFO] Cargando agrupación de curvas desde cache: /home/ec2-user/SageMaker/astro_transformer/src/fase2/../../data/train/grouped_data.pkl
✅ [INFO] Agrupación cargada desde cache. Total objetos: 55439
⏳ [INFO] Tiempo en agrupación de datos: 209.8 segundos
🚀 Procesando 55439 curvas en paralelo usando 20 CPUs...
⏳ [INFO] Tiempo en procesamiento paralelo: 81.7 segundos
🔋 [INFO] Curvas válidas tras filtrado: 55342

🔍 Realizando prueba rápida en características auxiliares...
✅ Sample 0 sin problemas: [ 0.4604889   0.45588233  0.45513864  0.71755386  0.5819672  -0.73028544
  0.43155103]
✅ Sample 1 sin problemas: [ 6.25626217  9.16911716  3.6892335   0.31717985  9.23360509 -0.52166147
 -0.62257893]
✅ Sample 2 sin problemas: [-0.36903283 -0.33455883 -0.34910278  0.22171369 -0.33606556 -0.84705968
  0.88971843]
✅ Sample 3 sin problemas: [ 1.66384515e+00  6.58061551e-04  5.05675775e+00 -2.81266735e-01
  3.39753782e-02  1.16336225e+01  7.70559508e+00]
✅ Sa

(<torch.utils.data.dataloader.DataLoader at 0x7f49fdfb16c0>,
 <torch.utils.data.dataloader.DataLoader at 0x7f49fdfb3280>)

**Script para generar los errores de clasificación con los IDs de objeto** 

Antes no los llevaban. La celda anterior fue para repetir el preprocesado ya con los IDs. 

In [2]:
import pandas as pd
import pickle
import torch
import torch.nn as nn
import torch.serialization
from torch.utils.data import DataLoader
import sys
import gc
import os
import argparse
import warnings
from pathlib import Path

# Añadir la raíz del proyecto al path
ROOT_DIR = Path.cwd().parent  # <- sube un nivel para alcanzar la raíz del proyecto
if str(ROOT_DIR) not in sys.path:
    sys.path.insert(0, str(ROOT_DIR))

# Ignorar solo los RuntimeWarning de numpy (como overflows en reduce)
warnings.filterwarnings("ignore", category=RuntimeWarning, module="numpy")

from src.fase2.script_2_transformer_fine_tuning_optimizado import AstroConformerClassifier as AstroConformerClassifier, evaluate

# Detectar dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"

# acciones para resolver los problemas de memoria
# 1. Liberar memoria
gc.collect()
torch.cuda.empty_cache()

# 2. Optimizar fragmentacion
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

torch.backends.cudnn.benchmark = True

def detectar_todos_los_errores(preds, true, ids, label_encoder):
    """
    Retorna un DataFrame con todos los errores de clasificación con clase real y predicha bien decodificadas.
    """
    df_labels = pd.DataFrame({
        "id_objeto": ids,
        "true_label": true,
        "pred_label": preds
    })

    # Decodificar nombres
    label_decoder = {v: k for k, v in label_encoder.items()}
    df_labels["clase_real"] = df_labels["true_label"].map(label_decoder)
    df_labels["clase_predicha"] = df_labels["pred_label"].map(label_decoder)

    # Filtrar errores
    df_errores = df_labels[df_labels["clase_real"] != df_labels["clase_predicha"]][
        ["id_objeto", "clase_real", "clase_predicha"]
    ].reset_index(drop=True)

    return df_errores

def detectar_yso_confundidas(preds, true, ids, label_encoder):
    """
    Retorna un DataFrame con los objetos cuya clase fue predicha como YSO pero no lo eran.
    """
    df_labels = pd.DataFrame({
        "id_objeto": ids,
        "true_label": true,
        "pred_label": preds
    })

    label_decoder = {v: k for k, v in label_encoder.items()}
    df_labels["clase_real"] = df_labels["true_label"].map(label_decoder)
    df_labels["clase_predicha"] = df_labels["pred_label"].map(label_decoder)

    df_yso_mal = df_labels[
        (df_labels["clase_predicha"] == "Young Stellar Object") &
        (df_labels["clase_real"] != "Young Stellar Object")
    ][["id_objeto", "clase_real", "clase_predicha"]].reset_index(drop=True)

    return df_yso_mal


# Cargar dataset y label encoder
from src.fase2.script_1_transformer_preprocessing_optimizado_2 import LightCurveDataset
#torch.serialization.add_safe_globals([LightCurveDataset])

val_dataset = torch.load("../data/train/val_dataset.pt", weights_only=False)

# Verificacion rapida
# Cargar un sample cualquiera
sample = val_dataset[0]
# Ver cuántos elementos contiene
print("Nº de elementos devueltos por __getitem__:", len(sample))
# Inspeccionar los elementos
for i, item in enumerate(sample):
    print(f"Elemento {i}: {type(item)}, shape o valor: {getattr(item, 'shape', item)}")

with open("../data/train/label_encoder.pkl", "rb") as f:
    label_encoder = pickle.load(f)
num_classes = len(label_encoder)
class_names = list(label_encoder.keys())

# Dataloader con batch pequeño
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=6, pin_memory=True, persistent_workers=True)

# Crear el modelo con la arquitectura esperada
args = argparse.Namespace(
    input_dim=1,
    in_channels=1,
    encoder_dim=256,
    hidden_dim=384,
    output_dim=num_classes,
    num_heads=8,
    num_layers=8,
    dropout=0.4, dropout_p=0.4,
    stride=32,
    kernel_size=3,
    norm="postnorm",
    encoder=["mhsa_pro", "conv", "conv"],
    timeshift=False,
    device=device
)
model = AstroConformerClassifier(args, num_classes=len(label_encoder), feature_dim=7)

# Cargar los pesos entrenados
state_dict = torch.load("../outputs/mejor_modelo_finetuned_optimizado2_features_segunda_vuelta.pt", map_location="cpu")
# Elimina el prefijo "_orig_mod." de las claves
new_state_dict = {k.replace("_orig_mod.", ""): v for k, v in state_dict.items()}
model.load_state_dict(new_state_dict)

# Pasar a GPU si está disponible
model = model.to(device)
model.eval()  # Muy importante: modo evaluación

criterion = nn.CrossEntropyLoss()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Evaluar con IDs
val_loss, preds, true, ids = evaluate(model, val_loader, criterion, device)

#####################################
# Mostrar el label encoder para asegurar consistencia (opcional si ya lo hiciste)
print("Contenido del label encoder:")
for key, value in label_encoder.items():
    print(f"{key}: {value}")
# Mapear IDs a clases reales desde el CSV original
df_debug = pd.read_csv("../data/train/debug_clases_codificadas.csv")
dict_clases_reales = dict(zip(df_debug["id"].astype(str), df_debug["clase_variable"]))
print("\n🔍 Verificación manual de los primeros errores:")
for i in range(10):
    pred_label = class_names[preds[i]]
    true_label = class_names[true[i]]
    object_id = str(ids[i])

    real_ref = dict_clases_reales.get(object_id, "NO_ENCONTRADO")

    print(f"ID: {object_id}")
    print(f" - Predicha: {pred_label}")
    print(f" - Real (según modelo): {true_label}")
    print(f" - Real (en CSV original): {real_ref}")
    print("---")
#####################################

# Detectar YSO mal clasificadas
df_yso_mal = detectar_yso_confundidas(preds, true, ids, label_encoder)
df_yso_mal.to_csv("../outputs/yso_clase_predicha_error.csv", index=False)
# Detectar todos los errores
df_todos_errores = detectar_todos_los_errores(preds, true, ids, label_encoder)
df_todos_errores.to_csv("../outputs/todos_los_errores.csv", index=False)

print(f"YSOs mal clasificadas detectadas: {len(df_yso_mal)}")
print(f"Total de errores detectados: {len(df_todos_errores)}")


Nº de elementos devueltos por __getitem__: 5
Elemento 0: <class 'torch.Tensor'>, shape o valor: torch.Size([25000])
Elemento 1: <class 'torch.Tensor'>, shape o valor: torch.Size([])
Elemento 2: <class 'torch.Tensor'>, shape o valor: torch.Size([25000])
Elemento 3: <class 'torch.Tensor'>, shape o valor: torch.Size([7])
Elemento 4: <class 'str'>, shape o valor: ASASSN-V J024641.95+724748.3
Contenido del label encoder:
Cataclysmic: 0
Delta Scuti: 1
Eclipsing Binary: 2
Irregular: 3
RR Lyrae: 4
Rotational: 5
Young Stellar Object: 6

🔍 Verificación manual de los primeros errores:
ID: ASASSN-V J024641.95+724748.3
 - Predicha: Eclipsing Binary
 - Real (según modelo): Eclipsing Binary
 - Real (en CSV original): Eclipsing Binary
---
ID: ASASSN-V J045112.25+255829.6
 - Predicha: RR Lyrae
 - Real (según modelo): RR Lyrae
 - Real (en CSV original): RR Lyrae
---
ID: ASASSN-V J023630.93+420129.6
 - Predicha: Delta Scuti
 - Real (según modelo): Delta Scuti
 - Real (en CSV original): Delta Scuti
---
ID

#### Verificar que las clases reales en el fichero de errores generado coincide con las clases reales codificadas durante el preprocesado

En pruebas anteriores vimos que no estaban igual en ambos sitios

#### SCRIPTS PARA EXTRAER LOS CASOS DE CURVAS DUDOSAS QUE SE PODRÍAN ELIMINAR DEL DATASET DE ENTRENAMIENTO

Sobre todo a raíz de ver que las YSO, a pesar de ser ahora la clase mayoritaria, es la que más confusiones está generando en el modelo, tanto por falsos positivos como por falsos negativos.

In [12]:
import pandas as pd

# Cargar CSV de todas las curvas (con clase real)
df_all = pd.read_csv("../data/train/debug_clases_codificadas.csv")

# Cargar errores donde se predijo YSO y no lo era (CASO A)
df_yso_fp = pd.read_csv("../outputs/yso_clase_predicha_error.csv")
ids_fp = set(df_yso_fp["id_objeto"])

# Imprimir algunos ejemplos de IDs falsos positivos y el total
print(f"🔍 Total de IDs falsos positivos (YSO predicha pero no real): {len(ids_fp)}")
# print("🔍 Ejemplos de IDs falsos positivos (YSO predicha pero no real):")
# for i, obj_id in enumerate(ids_fp):
#     if i < 5:  # Limitar a 5 ejemplos
#         print(f"ID {i+1}: {obj_id}")
#     else:
#         break

# Cargar errores generales de clasificacion (CASO B)
df_errores = pd.read_csv("../outputs/todos_los_errores.csv")

# Añadir ID si está disponible (debes tener otro CSV con índices → IDs si no lo incluiste)
# Aquí asumimos que ya tienes columna "id_objeto" cruzada
df_fn_yso = df_errores[
    df_errores["clase_real"] == "Young Stellar Object"
].copy()
ids_fn = set(df_fn_yso["id_objeto"])

# Imprimir algunos ejemplos de IDs falsos negativos y el total
print(f"🔍 Total de IDs falsos negativos (YSO real pero no predicha): {len(ids_fn)}")
# print("🔍 Ejemplos de IDs falsos negativos (YSO real pero no predicha):")
# for i, obj_id in enumerate(ids_fn):
#     if i < 5:  # Limitar a 5 ejemplos
#         print(f"ID {i+1}: {obj_id}")
#     else:
#         break

# 🔁 CURVAS QUE QUEREMOS DEPURAR = unión de ambas
ids_dudosos = ids_fp.union(ids_fn)

# Imrpimir algunos ejemplos de IDs dudosos
# print("🔍 Ejemplos de IDs dudosos:" )
# for i, obj_id in enumerate(ids_dudosos):
#     if i < 5:  # Limitar a 5 ejemplos
#         print(f"ID {i+1}: {obj_id}")
#     else:
#         break

# Filtrar todas las YSO reales del dataset original
df_yso_all = df_all[df_all["clase_variable"] == "Young Stellar Object"].copy()

# Marcar cuáles son dudosas
df_yso_all["dudosa"] = df_yso_all["id"].isin(ids_dudosos)

# Estadísticas
print(f"🔢 Total YSO reales: {len(df_yso_all)}")
print(f"⚠️ YSO dudosas detectadas: {df_yso_all['dudosa'].sum()}")

# Ordenar para dejar las seguras primero
df_yso_ordenadas = df_yso_all.sort_values(by="dudosa", ascending=True)

# Seleccionar 9000 curvas más confiables
df_yso_final = df_yso_ordenadas.head(9000).copy()

# Guardar resultado
#df_yso_final.to_csv("../data/train/ysos_9000_filtradas_seguras.csv", index=False)
#print("✅ Guardado en outputs/ysos_9000_filtradas_seguras.csv")


🔍 Total de IDs falsos positivos (YSO predicha pero no real): 794
🔍 Total de IDs falsos negativos (YSO real pero no predicha): 440
🔢 Total YSO reales: 9799
⚠️ YSO dudosas detectadas: 440


In [14]:
import pandas as pd

# 1. Cargar CSV de todas las curvas con su clase real
df_all = pd.read_csv("../data/train/debug_clases_codificadas.csv")

# 2. FALSOS NEGATIVOS: eran YSO pero el modelo no las predijo como tal
df_errores = pd.read_csv("../outputs/todos_los_errores.csv")
df_fn = df_errores[df_errores["clase_real"] == "Young Stellar Object"].copy()
df_fn["motivo_descarte"] = "YSO mal clasificada (FN)"
df_fn = df_fn[["id_objeto", "clase_real", "motivo_descarte"]]
df_fn.rename(columns={"clase_real": "clase_original"}, inplace=True)
# Imprimir total de FN
print(f"🔍 Total de FALSOS NEGATIVOS (YSO reales pero no predichas): {len(df_fn)}")

# 3. FALSOS POSITIVOS: no eran YSO pero el modelo las predijo como YSO
df_fp = pd.read_csv("../outputs/yso_clase_predicha_error.csv")
df_fp["motivo_descarte"] = "YSO predicha incorrectamente (FP)"
df_fp.rename(columns={"clase_real": "clase_original", "id": "id_objeto"}, inplace=True)
df_fp = df_fp[["id_objeto", "clase_original", "motivo_descarte"]]
# Imprimir total de FP
print(f"🔍 Total de FALSOS POSITIVOS (YSO predichas pero no reales): {len(df_fp)}")

# 4. Unir ambos
df_dudosos = pd.concat([df_fn, df_fp], ignore_index=True)

# 5. Verificar qué porcentaje representan
print(f"🔍 Total de curvas a eliminar: {len(df_dudosos)}")
print(df_dudosos["motivo_descarte"].value_counts())

# 6. Añadir columna auxiliar para posibles análisis
df_dudosos["origen"] = df_dudosos["id_objeto"].apply(
    lambda x: "ASASSN" if "ASASSN" in x else "ZTF" if "ZTF" in x else "TESS" if "TIC_" in x else "Otros"
)

# 7. Generar desglose por clase real
print("\n📊 Desglose por clase original (impacto en dataset):")
print(df_dudosos["clase_original"].value_counts())

# 8. Guardar CSV de curvas a eliminar
df_dudosos.to_csv("../data/train/curvas_a_eliminar_por_confusion_yso.csv", index=False)
print("✅ Guardado en: ../data/train/curvas_a_eliminar_por_confusion_yso.csv")


🔍 Total de FALSOS NEGATIVOS (YSO reales pero no predichas): 440
🔍 Total de FALSOS POSITIVOS (YSO predichas pero no reales): 794
🔍 Total de curvas a eliminar: 1234
motivo_descarte
YSO predicha incorrectamente (FP)    794
YSO mal clasificada (FN)             440
Name: count, dtype: int64

📊 Desglose por clase original (impacto en dataset):
clase_original
Young Stellar Object    440
Eclipsing Binary        207
Cataclysmic             186
Rotational              162
Delta Scuti              99
RR Lyrae                 93
Irregular                47
Name: count, dtype: int64
✅ Guardado en: ../data/train/curvas_a_eliminar_por_confusion_yso.csv


In [15]:
max_per_class_override={
    "Irregular": 9000,
    "Rotational": 9000,
    "Eclipsing Binary": 9000,
    "Delta Scuti": None, # 7.550 → TODAS
    "RR Lyrae": 9000, # 41.208 → TODAS NO
    "Young Stellar Object": None, # 9.809 → TODAS
    "Cataclysmic": None, # 2.080 → TODAS
    "White Dwarf": 0, # 0 → LA ELIMINAMOS
    "Variable": 0 # 0 → LA ELIMINAMOS
}
max_per_class = None  # Si no hay override, usamos el global

df_malas = pd.read_csv("../data/train/curvas_a_eliminar_por_confusion_yso.csv")
ids_a_excluir = set(df_malas["id_objeto"].astype(str))
# Contar por clase original
ids_a_excluir_por_clase = df_malas.groupby("clase_original")["id_objeto"].nunique().to_dict()
print(f"\U0001F4C2 [INFO] IDs a excluir por filtrado: {len(ids_a_excluir)}")
print(f"\U0001F4C2 [INFO] Exclusiones por clase: {ids_a_excluir_por_clase}")

# Ajustar límites de clase
if max_per_class_override:
    for clase, n_excluir in ids_a_excluir_por_clase.items():
        if clase in max_per_class_override and max_per_class_override[clase] is not None:
            nuevo_limite = max(0, max_per_class_override[clase] - n_excluir)
            print(f"   - Ajustando max_per_class_override[{clase}] de {max_per_class_override[clase]} a {nuevo_limite}")
            max_per_class_override[clase] = nuevo_limite
elif max_per_class is not None:
    # Si no hay override, solo hay un límite global
    total_excluir = sum(ids_a_excluir_por_clase.values())
    if max_per_class is not None:
        nuevo_limite = max(0, max_per_class - total_excluir)
        print(f"   - Ajustando max_per_class de {max_per_class} a {nuevo_limite}")
        max_per_class = nuevo_limite

📂 [INFO] IDs a excluir por filtrado: 1234
📂 [INFO] Exclusiones por clase: {'Cataclysmic': 186, 'Delta Scuti': 99, 'Eclipsing Binary': 207, 'Irregular': 47, 'RR Lyrae': 93, 'Rotational': 162, 'Young Stellar Object': 440}
   - Ajustando max_per_class_override[Eclipsing Binary] de 9000 a 8793
   - Ajustando max_per_class_override[Irregular] de 9000 a 8953
   - Ajustando max_per_class_override[RR Lyrae] de 9000 a 8907
   - Ajustando max_per_class_override[Rotational] de 9000 a 8838
